In [1]:
# Nativos
import random as rn
import os
import sys
import gc
import time

#calculo
import numpy as np
import pandas as pd
import scipy

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

SEED = int(time.time() / 1000)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

from utils_alertas import *
print(SEED)

EJEMPLO:
202104 mes cerrado de la cosecha de 202104 [202011, 202012, 202101, 202102, 202103, 202104] [202102, 202103, 202104] 
 historia 
 [202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104] 
 ---------------------------------------------------------------------------------------------------- 
 [202105, 202106] 
 ---------------------------------------------------------------------------------------------------- 
 [202005, 202006]
EJEMPLO:
202003 mes cerrado de la cosecha de 202003 [201910, 201911, 201912, 202001, 202002, 202003] [202001, 202002, 202003] 
 historia 
 [201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912, 202001, 202002, 202003] 
 ---------------------------------------------------------------------------------------------------- 
 [202004, 202005] 
 ---------------------------------------------------------------------------------------------------- 
 [201904, 201905]
1623258


In [2]:
data = pd.read_csv('AVANCE_6.csv', dtype={
    'COD_UNICO': str,
    'PERIODO': str
})
data.head()

,_6_cut_up_F_Septile,ADEX_PROM_U6M_ACTUAL_IMP_OVER_12M_PASADO_MENOR_0.55,_aux_TRX_CAIDA_5PERCENT_COBRANZAS__MES_ANT,ACTIVIDAD,_300_cut_up_ROTACION_INVENTARIOS_CEF_SITUACION,_600000.0_cut_up_RIESGO,TRX_SUBIDA_SEGUIDA_U3M_CASH_IN,TIENE_DISMINUCION__CASHIN_50PERCENT_A_MAS_U12M,MAX_DIAS_ATRASO_BCPSF_ULT_RCC_x,TIENE_PRESTAMOS_Y_LEASING_VIGENTES_SF_ULT_RCC,...,_3_cut_up_NUM_DOCS_PROT_PROMEDIO_U9M,_2_cut_up_NUM_DOCS_PROT_U6M,_2_cut_up_NUM_DOCS_LAB_U1M,_0.4_cut_up_PERCENT_GAR_OTROS_BANCOS_PRIN,_5_cut_up_MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x,_0.4_cut_down_minimo_percent_auto_hipo_u12m,departamento_preocupa,_1_cut_up_NUM_DOCS_TRIB_U6M,_0.5_cut_down_SOW_OTROSBCOS_MAX_historia,_1_cut_down_NUM_BANCOS_DEBT_MAYOR_DE_U3M
0,0.0,0.0,0.0,NaN,1.0,0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0,0.0,0.0,1
1,0.0,0.0,0.0,Comercio,1.0,0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1,0.0,0.0,0
2,0.0,0.0,0.0,NaN,0.0,1,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0,0.0,0.0,1
3,0.0,0.0,0.0,NaN,0.0,0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0,0.0,0.0,0
4,1.0,0.0,0.0,NaN,0.0,0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0,0.0,0.0,0


In [3]:
'COD_UNICO' in data.columns, 'PERIODO' in data.columns

(True, True)

In [4]:
print(data.shape)
data = filtro_banca_mediana(data)
print(data.shape)

(24014, 4042)
(4829, 4042)


In [5]:
pd.crosstab(
    data['PERIODO'], data['ZONAL']
)

ZONAL,BEL ZONAL 2
PERIODO,
202009,834
202010,807
202011,788
202012,803
202101,799
202102,798


### CREACION DE COLUMNAS A MEZCLAR

In [6]:
cols_combine = [col for col in data.columns if set(data[col].unique()) == {1, 0} if '1_and_1' not in col]
rn.shuffle(cols_combine)
print(len(cols_combine))

2735


In [7]:
cols_combine = [col for col in cols_combine if 'SCOTIA' not in col.upper() \
                                 and 'BCP' not in col.upper() \
                                 and 'BBVA' not in col.upper() \
                                 and 'BIF' not in col.upper() \
                                 and 'COVID' not in col.upper() 
    ]
print(len(cols_combine))

2651


In [8]:
for col in ['cambio_seguir_covid'] + [col for col in data.columns if 'target' in col.lower()]:
    if col in cols_combine:
        cols_combine.remove(col)

len(cols_combine)

2650

### GENERACION DE ALERTAS

In [9]:
existentes = pd.read_csv('COMBINACIONES_MEDIANA_LIMA.csv').sort_values(by=['SCORE'], ascending=False)
existentes.head(5)
#[a, b, None, coincidencias, acertados, acertados/coincidencias]

,VAR_1,VAR_2,VAR_3,CASOS,CORRECTOS,SCORE
16729,_4_cut_up_RMET_9_METODIZADO_CEF_SITUACION,_1_cut_up_NUM_DOCS_LAB_U12M,NaN,6,6.0,1.0
6352,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.2,DEBT_CAIDA_SEGUIDA_MTOSALDO_IBK_U3M_SF,NaN,5,5.0,1.0
12059,_2_cut_up_NUM_MESES_ANTERIORES_CON_IMPORTE_LAB...,_2_cut_up_RMET_9_METODIZADO_CEF_SITUACION,NaN,5,5.0,1.0
12710,_4_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF,_0_cut_up_TIENE_TENDENCIA_CRECIENTE_IMPORTE_IM...,NaN,6,6.0,1.0
11985,_3_cut_up_NUM_MESES_ANTERIORES_CON_IMPORTE_LAB...,_aux_TRX_CAIDA_30PERCENT_OPERACIONES_MESA__U11M,NaN,5,5.0,1.0


In [10]:
try:
    existentes['union'] = existentes[['VAR_1', 'VAR_2', 'VAR_3']].apply(
        lambda _: '1_and_1_{}___{}'.format(_[0], _[1]) if pd.isnull(_[2]) else '1_and_1_and_1_{}___{}____{}'.format(_[0], _[1], _[2]), 
        axis=1
    )
    listado = list(existentes['union'].unique())
    len(listado)
except:
    listado = []
    
len(listado)

18995

In [11]:
import random as rn
import itertools

display(pd.crosstab(
    data[col_target].fillna(-1), data['PERIODO']
))

aucc = {}
struct = []

data = data[data[col_target].notnull()][cols_combine + ['COD_UNICO', col_target, 'PERIODO']]
data.shape

PERIODO,202009,202010,202011,202012,202101,202102
target_FINAL_fusion_atraso_feve_2Mbueno,,,,,,
-1.0,269,239,221,204,197,193
0.0,515,519,522,552,554,553
1.0,50,49,45,47,48,52


(3506, 2653)

In [12]:
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_MEDIANA_LIMA.csv', index=False)
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_GRAN_EMPRESA_LIMA.csv', index=False)
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_S1_PROVINCIA.csv', index=False)
#pd.DataFrame([], columns=['VAR_1', 'VAR_2', 'VAR_3', 'CASOS', 'CORRECTOS', 'SCORE']).to_csv('COMBINACIONES_S2_PROVINCIA.csv', index=False)

In [13]:
over_interes = [col for col in data.columns if ('ADEX' in col.upper() and '12M' in col.upper()) and '1_and_1' not in col]
len(over_interes)

199

In [14]:
#print(over_interes)

In [ ]:
from datetime import datetime

tiempo_inicio = time.time()
nn = len(struct)

while True:
    x = rn.choice(over_interes)
    y = rn.choice(cols_combine)
    z = rn.choice(cols_combine)
    
    col_evaluate = '1_and_1_and_1_{}___{}____{}'.format(x, y, z)
    
    if col_evaluate not in listado and x != y and x!= z and y!= z:
        
        listado.append(col_evaluate)
        
        lambda_basico = lambda _:  _[0] >= 1 and _[1] >= 1 and _[2] >= 1
        data[col_evaluate] = data[[x, y, z]].apply(lambda_basico, axis=1).astype(int)
        #data_play_1[col_evaluate] = data_play_1[[x, y, z]].apply(lambda_basico, axis=1).astype(int)
        
        coincidencias = len(set(data[data[col_evaluate] == 1]['COD_UNICO'].unique()))
        acertados = len(set(data[(data[col_evaluate] == 1) & (data[col_target] == 1)]['COD_UNICO'].unique()))
        
        if acertados > 5 and acertados/coincidencias > 0.3:  # cañitos
            aucc[col_evaluate] = acertados/coincidencias
            struct.append(
                [x, y, z, coincidencias, acertados, acertados/coincidencias]
            )
            #tiempo_inicio = time.time()
            nn = len(struct)
            #print(struct[-1])
            
        ### Generado pares de 2 
        for a, b in list(itertools.combinations([x, y, z], 2)):
            
            col_evaluate = '1_and_1_{}___{}'.format(a, b)
            if col_evaluate not in listado:
                lambda_basico = lambda _:  _[0] >= 1 and _[1] >= 1
                data[col_evaluate] = data[[a, b]].apply(lambda_basico, axis=1).astype(int)
                #data_play_1[col_evaluate] = data_play_1[[a, b]].apply(lambda_basico, axis=1).astype(int)
                
                coincidencias = len(set(data[data[col_evaluate] == 1]['COD_UNICO'].unique()))
                acertados = len(set(data[(data[col_evaluate] == 1) & (data[col_target] == 1)]['COD_UNICO'].unique()))
                
                if acertados > 5 and acertados/coincidencias > 0.3: #cañitos
                    aucc[col_evaluate] = acertados/coincidencias
                    struct.append(
                        [a, b, None, coincidencias, acertados, acertados/coincidencias]
                    )
                    tiempo_inicio = time.time()
                    nn = len(struct)
                    #print(struct[-1])
        
        if len(struct) >= 5:
            pd.DataFrame(struct).to_csv('COMBINACIONES_MEDIANA_LIMA.csv', header=False, index=False, mode='a')
            print("/"*25, str(datetime.now()).split('.')[0])
            struct = []
        
        if time.time() - tiempo_inicio >= 3600 and nn == len(struct):
            print("Mas de 1 hora sin nueva combinacion de alerta")
            break

///////////////////////// 2021-06-09 12:12:18
///////////////////////// 2021-06-09 12:12:40
///////////////////////// 2021-06-09 12:13:05
///////////////////////// 2021-06-09 12:13:40
///////////////////////// 2021-06-09 12:14:03
///////////////////////// 2021-06-09 12:14:28
///////////////////////// 2021-06-09 12:14:40
///////////////////////// 2021-06-09 12:14:56
///////////////////////// 2021-06-09 12:15:26
///////////////////////// 2021-06-09 12:15:56
///////////////////////// 2021-06-09 12:16:41
///////////////////////// 2021-06-09 12:16:58
///////////////////////// 2021-06-09 12:17:35
///////////////////////// 2021-06-09 12:18:32
///////////////////////// 2021-06-09 12:19:28
///////////////////////// 2021-06-09 12:20:11
///////////////////////// 2021-06-09 12:20:49
///////////////////////// 2021-06-09 12:21:54
///////////////////////// 2021-06-09 12:22:17
///////////////////////// 2021-06-09 13:18:52
///////////////////////// 2021-06-09 13:19:13
///////////////////////// 2021-06-